# **Introduction to Building Agents**

In this notebook we are going to first see how to build a basic agent.

Later we will use LangGraph to build powerful agents.

## **Step 1 - Initializing a Large Language Model**

In [1]:
from langchain_openai import ChatOpenAI

/Users/kanavbansal/Developer/.env_jupyter/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# Setup API Key

f = open('keys/.openai_api_key.txt')

OPENAI_API_KEY = f.read()

In [3]:
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY,
                 model="gpt-4o-mini",
                 temperature=0.0)

llm.invoke("Hi")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 8, 'total_tokens': 18, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bd83329f63', 'finish_reason': 'stop', 'logprobs': None}, id='run-5422a792-c580-416b-b443-3638e1e54bc4-0', usage_metadata={'input_tokens': 8, 'output_tokens': 10, 'total_tokens': 18, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## **Step 2 - Initializing a Tool**

In [4]:
from langchain_core.tools import tool
from datetime import datetime

@tool
def current_datetime():
    """This function tells the current date and time."""
    current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    return f"The current date and time is {current_datetime}"

current_datetime.invoke({})

'The current date and time is 2025-02-13 14:51:10'

## **Step 3 - Init an Agent (i.e. Binding the Tool with a LLM)**

In [5]:
llm_with_tool = llm.bind_tools([current_datetime])

llm_with_tool

RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x10c8e6b20>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x10c8c3cd0>, root_client=<openai.OpenAI object at 0x10c8a77f0>, root_async_client=<openai.AsyncOpenAI object at 0x10c8e6b80>, model_name='gpt-4o-mini', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'current_datetime', 'description': 'This function tells the current date and time.', 'parameters': {'properties': {}, 'type': 'object'}}}]}, config={}, config_factories=[])

In [6]:
response = llm_with_tool.invoke("Hi")

print(f"Content: {response.content}")
print(f"Tool Call: {response.tool_calls}")

Content: Hello! How can I assist you today?
Tool Call: []


In [7]:
response = llm_with_tool.invoke("What is the current date?")

print(f"Content: {response.content}")
print(f"Tool Call: {response.tool_calls}")

Content: 
Tool Call: [{'name': 'current_datetime', 'args': {}, 'id': 'call_pQ9MIOiVKMgsFx3pG47dHYlL', 'type': 'tool_call'}]


## **Step 4 - Executing the Agent**

In [8]:
dict_with_tool_names = {
    "current_datetime": current_datetime,
}

def custom_agent_executor(response):
    if response.tool_calls:
        for tool_calls in response.tool_calls:
            tool_name = tool_calls["name"]
            tool_args = tool_calls["args"]
            tool_func = dict_with_tool_names[tool_name]
            return tool_func.invoke(tool_args)
    else:
        return response.content

In [9]:
custom_agent_executor(llm_with_tool.invoke("Hi"))

'Hello! How can I assist you today?'

In [10]:
custom_agent_executor(llm_with_tool.invoke("What is the time?"))

'The current date and time is 2025-02-13 14:51:20'

# **A Better Implementation**

## **Step 1 - Init a LLM**

## **Step 2 - Init a Tool**

In [11]:
from datetime import datetime

def current_datetime():
    """This function tells the current date and time."""
    current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    return f"The current date and time is {current_datetime}"

current_datetime()

'The current date and time is 2025-02-13 14:51:37'

## **Step 3 - Init an Agent**

In [12]:
# ! pip install langgraph

In [13]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model=llm, tools=[current_datetime])

**IMPORTANT:** Note that we are passing in the **llm**, not **llm_with_tools**. That is because **create_react_agent** will call **.bind_tools** for us under the hood.

## **Step 4 - Execute the Agent**

In [14]:
response = agent_executor.invoke({"messages": "Hi"})

response

{'messages': [HumanMessage(content='Hi', additional_kwargs={}, response_metadata={}, id='2ffd2555-552b-4636-abc5-60bb123c54a0'),
  AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 41, 'total_tokens': 52, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bd83329f63', 'finish_reason': 'stop', 'logprobs': None}, id='run-374e8030-b6a1-48b3-9a40-25a9b4030f79-0', usage_metadata={'input_tokens': 41, 'output_tokens': 11, 'total_tokens': 52, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}

In [15]:
response = agent_executor.invoke({"messages": "What is the time?"})

response

{'messages': [HumanMessage(content='What is the time?', additional_kwargs={}, response_metadata={}, id='3e79aa20-e204-47c7-8d0a-5248d2acdc32'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_CfYEaA4KFn0n6AxQAzM98cSj', 'function': {'arguments': '{}', 'name': 'current_datetime'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 45, 'total_tokens': 56, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-e4d17a57-2af9-4804-a4c3-770f8685bead-0', tool_calls=[{'name': 'current_datetime', 'args': {}, 'id': 'call_CfYEaA4KFn0n6AxQAzM98cSj', 'type': 'tool_call'}], usage_metadata={'input_tokens': 45, 'output_tokens': 11, 

## **Step 5 - Pretty Print**

In [25]:
def printer(input_prompt):
    for s in agent_executor.stream(input_prompt, stream_mode="values"):
        msg = s["messages"][-1]
        if isinstance(msg, tuple):
            print("tuple:", msg)
        else:
            msg.pretty_print()

In [26]:
printer(input_prompt={"messages": "hi"})

================================ Human Message =================================

hi
================================== Ai Message ==================================

Hello! How can I assist you today?


In [27]:
printer(input_prompt={"messages": "time?"})

================================ Human Message =================================

time?
================================== Ai Message ==================================
Tool Calls:
  current_datetime (call_mM9vGAiVnymCpdw93OxTihDo)
 Call ID: call_mM9vGAiVnymCpdw93OxTihDo
  Args:
================================= Tool Message =================================
Name: current_datetime

The current date and time is 2025-02-13 14:59:25
================================== Ai Message ==================================

The current date and time is February 13, 2025, at 14:59:25.
